## bleu score

In [1]:
%matplotlib inline 

In [2]:
run -i bleu.py

/Users/bilals01/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
  
import re
import random
import collections
import torch
import numpy as np
import nltk

#%%
with open("/Users/bilals01/Documents/NLP/project/quasi-rnn-run-12-09/evaluation/source_valid.txt", "r", encoding="utf8") as f:
    
    source = []
    
    for i, line in enumerate(f):
        
        line = line.split('<JOIN>')
        source.append(line[0])
        
  
    
with open("/Users/bilals01/Documents/NLP/project/quasi-rnn-run-12-09/evaluation/target_valid.txt", "r", encoding="utf8") as f:
    
    targets = []
    
    for i, line in enumerate(f):
        
        line = line.split('<JOIN>')
        targets.append(line[0])
        
        
    
with open("/Users/bilals01/Documents/NLP/project/quasi-rnn-run-12-09/evaluation/pred_valid_m1.txt", "r", encoding="utf8") as f:
    
    predictions1 = []
    s2=[]
    
    for i, line in enumerate(f):
        s1=(line.split('  '))
        #print(s1)
        for k, word in enumerate(s1):
            s2.append(" ".join(["".join(w.split(" ")) for w in word.split("  ")]))
        predictions1.append(" ".join(s2))
        s2=[]
        
        
with open("/Users/bilals01/Documents/NLP/project/quasi-rnn-run-12-09/evaluation/pred_valid_m2.txt", "r", encoding="utf8") as f:
    
    predictions2 = []
    s2=[]
    
    for i, line in enumerate(f):
        s1=(line.split('  '))
        #print(s1)
        for k, word in enumerate(s1):
            s2.append(" ".join(["".join(w.split(" ")) for w in word.split("  ")]))
        predictions2.append(" ".join(s2))
        s2=[]
         

In [4]:
def normalizeString(s):
    s = s.lower().strip()
    s = re.sub(r"([.!?])", r" ", s) #separates punctuation from the word
    #s = re.sub(r"([.!?])", r" \1", s) #separates punctuation from the word
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s) #strips anything that isn't a character of punctuation
    
    return s

In [5]:
#get rid of /n 
source_u=[]
for i, word in enumerate(source): 
    source_u.append(normalizeString(word.strip()))
    
target_u=[]
for i, t in enumerate(targets):
    target_u.append(normalizeString(t.strip()))
    
pred_u1=[]
for i, p in enumerate(predictions1):
    pred_u1.append(normalizeString(p.strip()))  

pred_u2=[]
for i, p in enumerate(predictions2):
    pred_u2.append(normalizeString(p.strip()))  
   

In [7]:
print(len(source))
print(len(targets))
print(len(predictions1))
print(len(predictions2))


2052
2052
2052
2052


In [200]:
print(target_u[8950])
print('')
print(pred_u[8950])

and so she went to dance every year on the first of may and danced until late and probably drank a glass of punch too much once and ended up falling and broke you can see it here on the right side her leg her femoral neck 

and so she is she was the first may first go to the dance and dancing and probably once a glass of bowl and probably once a glass of bowl that you see the leg broke the ginger that you see the leg broke the ginger that you see the leg broke the ginger 


In [8]:
#bleu score
a=moses_multi_bleu(pred_u1, target_u, lowercase=False)
b=moses_multi_bleu(pred_u2, target_u, lowercase=False)

print(a)
print(b)

11.39
13.5


In [202]:
with open("/Users/bilals01/Documents/NLP/project/quasi-rnn-run-12-09/evaluation/compounds.txt", "r", encoding="utf8") as comp:
    compound_words = []
    
    for i,line in enumerate(comp):        
        compound_words.append(line.strip())

In [203]:
#experiment-1: find the sentences where the compound words occurs and  
#find the  bleu score for those
pred_comp=[]
target_comp=[]

for i in range(len(pred_u)):
    for w in source_u[i].split():
        if w in compound_words:
            pred_comp.append(pred_u[i])
            target_comp.append(target_u[i])
            break
                  


In [204]:
blue_comp=moses_multi_bleu(pred_comp, target_comp, lowercase=False)

print(blue_comp)

12.1


In [205]:
#experiment-2: find the sentences where the compound words occurs and 
#count the number of words in each sentence.
target_length=[]
source_length=[]
pred_length=[]

t=[]
s=[]
p=[]

for i in range(len(pred_u)):
    for w in source_u[i].split():
        if w in compound_words:
            target_length.append(len(target_u[i].split()))
            source_length.append(len(source_u[i].split())) 
            pred_length.append(len(pred_u[i].split())) 

            t.append(target_u[i])
            s.append(source_u[i])
            p.append(pred_u[i])

            break
         

print(sum(target_length) / float(len(target_length)))
print(sum(source_length) / float(len(source_length)))
print(sum(pred_length) / float(len(pred_length)))


23.807193396226417
23.28183962264151
28.77063679245283


In [206]:
import numpy as np
from scipy import stats 

t_l = np.array(target_length)
s_l = np.array(source_length)
p_l = np.array(pred_length)

print(stats.sem(t_l))
print(stats.sem(s_l))
print(stats.sem(p_l))




0.353135948518
0.346535100124
0.376896965315


In [208]:
#not using this anymore

target_noncomp_length=[]
source_noncomp_length=[]
pred_noncomp_length=[]

#find the non compound sentences 
for i in range(len(target_u)):
    for sentence in target_u:
        if sentence in t:
            break
        else:
            target_noncomp_length.append(len(target_u[i].split()))
        
for i in range(len(source_u)):
    for sentence in source_u:
        if sentence in s:
            break
        else:
            source_noncomp_length.append(len(source_u[i].split()))
            
for i in range(len(pred_u)):
    for sentence in pred_u:
        if sentence in s:
            break
        else:
            pred_noncomp_length.append(len(pred_u[i].split()))
 
 

In [ ]:
print(sum(target_noncomp_length) / float(len(target_noncomp_length)))
print(sum(source_noncomp_length) / float(len(source_noncomp_length)))
print(sum(pred_noncomp_length) / float(len(pred_noncomp_length)))

In [211]:
#overall length
overall_source_length=[]
overall_target_length=[]
overall_pred_length=[]


for i in range(len(source_u)):
    overall_source_length.append(len(source_u[i].split()))
    overall_target_length.append(len(target_u[i].split()))
    overall_pred_length.append(len(pred_u[i].split()))

            
print(sum(overall_source_length) / float(len(overall_source_length)))
print(sum(overall_target_length) / float(len(overall_target_length)))
print(sum(overall_pred_length) / float(len(overall_pred_length)))

t_l = np.array(overall_target_length)
s_l = np.array(overall_source_length)
p_l = np.array(overall_pred_length)

print(stats.sem(t_l))
print(stats.sem(s_l))
print(stats.sem(p_l))



16.182850711142944
16.691190013301956
20.52174357924895
0.119243033233
0.117186928627
0.144922862633


In [223]:
#compare long vs short sentences
#use 10 as cutoff

long_sent_pred=[]
long_sent_target=[]
short_sent_pred=[]
short_sent_target=[]

for i in range(len(source_u)):
    if len(source_u[i])>100:
        long_sent_pred.append(pred_u[i])
        long_sent_target.append(target_u[i])
    else:
        short_sent_pred.append(pred_u[i])
        short_sent_target.append(target_u[i])



In [224]:
blue_long=moses_multi_bleu(long_sent_pred, long_sent_target, lowercase=False)
print(blue_long) 

blue_short=moses_multi_bleu(short_sent_pred, short_sent_target, lowercase=False)
print(blue_short)

12.4
13.79
